___

<p style="text-align: center;"><img src="https://docs.google.com/uc?id=1lY0Uj5R04yMY3-ZppPWxqCr5pvBLYPnV" class="img-fluid" 
alt="CLRSWY"></p>

## <p style="background-color:#FDFEFE; font-family:newtimeroman; color:#9d4f8c; font-size:100%; text-align:center; border-radius:10px 10px;">WAY TO REINVENT YOURSELF</p>

## <p style="background-color:#FDFEFE; font-family:newtimeroman; color:#060108; font-size:200%; text-align:center; border-radius:10px 10px;">DA & DVwPY</p>

## <p style="background-color:#FDFEFE; font-family:newtimeroman; color:#060108; font-size:200%; text-align:center; border-radius:10px 10px;">The Exploratory Data Analysis (EDA) Project</p>

<img src=https://i.ibb.co/wJW61Y2/Used-cars.jpg width="700" height="200">

## <p style="background-color:#FDFEFE; font-family:newtimeroman; color:#060108; font-size:200%; text-align:center; border-radius:10px 10px;">AutoScout Car Price Prediction EDA</p>



## Introduction
Welcome to "***AutoScout Exploratory Data Analysis (EDA) Project***". This is the project of completing ***Data Analysis & Data Visualization*** Courses and a successful hand-over is mandatory for getting certification for both courses. **Auto Scout** data used in this project scraped from the Website of an online car trading company in 2022, and contains many features of 13 different car makes including 594 models. In this project, you will have the opportunity to apply many commonly used algorithms for Data Cleaning and Exploratory Data Analysis by using a variety of Python libraries, such as Numpy, Pandas, Matplotlib, Seaborn, Scipy, and then you will get a clean dataset for your analysis and pretictive modelling in Machine Learning Path. So you will have the chance to use all the skills you have already learned in the Data Analysis and Visualization courses.

**``In this context, the project consists of 3 parts in general:``**
* **The first part** is related to ``'Data Cleaning'``. It deals with Incorrect Headers, Incorrect Format, Anomalies, and Dropping useless columns.
* **The second part** is related to ``'Filling Data'``, in other words 'Imputation'. It deals with Missing Values. Categorical to numeric transformation, Encoding, is done as well.
* **The third part** is related to ``'Handling Outliers of Data'`` via Visualization libraries. So, some insights will be extracted.

**``SPECIAL NOTE:``**  However, you are ``free to create your own style``. You do ``NOT`` have to stick to the steps above. Nevertheless, we, the DA & DV instructors, recommend you study each part separately to create a source notebook for your further studies. 

In order to build your Portfolio in terms of the GitHub account, you need to push your solution file up to your own repository.

Please after solving the task, commit your notebook to GitHub and submit its link to LMS page where the project is settled down.

**BE NOTED:** Please ``DO NOT FORGET`` to click the submit button.

### Some Reminders on Exploratory data analysis (EDA)

Exploratory data analysis (EDA) is an especially important activity in the routine of a data analyst or scientist. It enables an in depth understanding of the dataset, define or discard hypotheses and create predictive models on a solid basis. It uses data manipulation techniques and several statistical tools to describe and understand the relationship between variables and how these can impact business. By means of EDA, we can obtain meaningful insights that can impact analysis under the following questions (If a checklist is good enough for pilots to use every flight, it’s good enough for data scientists to use with every dataset).
1. What question are you trying to solve (or prove wrong)?
2. What kind of data do you have?
3. What’s missing from the data?
4. Where are the outliers?
5. How can you add, change or remove features to get more out of your data?

**``Exploratory data analysis (EDA)``** is often an **iterative brainstorming process** where you pose a question, review the data, and develop further questions to investigate before beginning model development work. The image below shows how the brainstorming phase is connected with that of understanding the variables and how this in turn is connected again with the brainstorming phase.<br>

<img src=https://i.ibb.co/k0MC950/EDA-Process.png width="300" height="100">

[Image Credit: Andrew D.](https://towardsdatascience.com/exploratory-data-analysis-in-python-a-step-by-step-process-d0dfa6bf94ee)



In [1]:
# jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import regex as re
pd.options.display.max_colwidth = 20
pd.options.display.max_columns = None
import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

In [2]:
def def_show_nulls(data):
    
    '''
    
    This function plots missing values for each column by observation in the dataset.
    
    ''' 
    plt.figure(figsize=(10, 5))

    sns.displot(data=data.isnull().melt(value_name="missing"),
                y="variable",
                hue="missing",
                multiple="fill",
                height=9.25)

    plt.axvline(0.2, color="r")
    plt.show()
    
# ---------------------------------------------------------------------------------------------------------------#

def def_col_name(x):
    '''
    
    This function is made to change incompatible column names.
    
    '''
    col_dict = {}
    for col in x.columns:
        new_col = col.strip().replace(' & ','_').replace(' ','_').replace('-','_').replace('CO₂','co2').replace('(','').replace(')','').lower()
        col_dict[col]= new_col
        x.rename(columns=col_dict, inplace=True)
        
# ---------------------------------------------------------------------------------------------------------------#

def def_col_val_analysis(x):
    '''
    
    This function was created to help you figure out what type of data is in the column as 
    it performs operations on each column.
    
    '''
    for col in x.columns:
        t_list = []
        for val in x[col]:
            t_list.append(type(val))
        print(pd.Series(t_list).value_counts())
        print(col+'\n')
        
# ---------------------------------------------------------------------------------------------------------------#

def def_clean_column(x):
    for i in range(0,len(x)):
        if x[i] != 'Unknown':
            x[i] = x[i][0].replace(',','').strip()
            
# ---------------------------------------------------------------------------------------------------------------#

def def_clean_column1(x):
    for i in range(len(x)):
        if x[i] != 'Unknown':
            x[i] = x[i][0].strip().replace(',','')

# ---------------------------------------------------------------------------------------------------------------#

def assign_consumption_value(df, i, idx, consumption_type):
    num = df.fuel_consumption[i][idx][0].split()[0]
    if consumption_type == '(city)':
        df.consumption_city[i] = num
    elif consumption_type == '(country)':
        df.consumption_country[i] = num
    elif consumption_type == '(comb.)':
        df.consumption_comb[i] = num

# ---------------------------------------------------------------------------------------------------------------#
        
def def_fill_miss(data, col_num, agg_func, col_target, first_col, second_col=None, third_col=None):
    '''
    
    This function is created to fill in missing data according to similar properties of other columns.
    
    '''
    indexes = data[data[col_target].isnull()==True].index
    
    if agg_func == 'mean':
        if col_num == 1:
            for i in indexes:
                data[col_target][i] = data[data[first_col]==data[first_col][i]][col_target].mean()
                if isinstance(data[col_target][i], float) and np.isnan(data[col_target][i]):
                    data[col_target][i] = data[col_target].mean()
        elif col_num == 2:
            for i in indexes:
                data[col_target][i] = data[(data[first_col]==data[first_col][i]) & (data[second_col]==data[second_col][i])][col_target].mean()
                if isinstance(data[col_target][i], float) and np.isnan(data[col_target][i]):
                    data[col_target][i] = data[data[first_col]==data[first_col][i]][col_target].mean()
                    if isinstance(data[col_target][i], float) and np.isnan(data[col_target][i]):
                        data[col_target][i] = data[col_target].mean()
        elif col_num == 3:
            for i in indexes:
                data[col_target][i] = data[(data[first_col]==data[first_col][i]) & (data[second_col]==data[second_col][i]) & (data[third_col]==data[third_col][i])][col_target].mean()
                if isinstance(data[col_target][i], float) and np.isnan(data[col_target][i]):
                    data[col_target][i] = data[(data[first_col]==data[first_col][i]) & (data[second_col]==data[second_col][i])][col_target].mean()    
                    if isinstance(data[col_target][i], float) and np.isnan(data[col_target][i]):
                        data[col_target][i] = data[data[first_col]==data[first_col][i]][col_target].mean()
                        if isinstance(data[col_target][i], float) and np.isnan(data[col_target][i]):
                            data[col_target][i] = data[col_target].mean()
                    
    elif agg_func == 'mode':
        if col_num == 1: 
            for i in indexes:
                a = data[data[first_col]==data[first_col][i]][col_target].mode()
                if len(a) > 0:
                    data[col_target][i] = a[0]
                else:
                    data[col_target][i] = data[col_target].mode()[0]
        elif col_num == 2:
            for i in indexes:
                a = data[(data[first_col]==data[first_col][i]) & (data[second_col]==data[second_col][i])][col_target].mode()
                if len(a) > 0:
                    data[col_target][i] = a[0]
                else:
                    aa = data[data[first_col]==data[first_col][i]][col_target].mode()
                    if len(aa) > 0:
                        data[col_target][i] = aa[0]
                    else:
                        data[col_target][i] = data[col_target].mode()[0]
        elif col_num == 3:
            for i in indexes:
                a = data[(data[first_col]==data[first_col][i]) & (data[second_col]==data[second_col][i]) & (data[third_col]==data[third_col][i])][col_target].mode()
                if len(a) > 0:
                    data[col_target][i] = a[0]
                else:
                    aa = data[(data[first_col]==data[first_col][i]) & (data[second_col]==data[second_col][i])][col_target].mode()
                    if len(aa) > 0:
                        data[col_target][i] = aa[0]
                    else:
                        aaa = data[data[first_col]==data[first_col][i]][col_target].mode()
                        if len(aaa) > 0:
                            data[col_target][i] = aaa[0]
                        else:
                            data[col_target][i] = data[col_target].mode()[0]

In [3]:
df0 = pd.read_json('as24_cars.json')
df = df0.copy()
df.fillna('Unknown', inplace=True)
def_col_name(df)

# PART- 1 `( Data Cleaning )`

In [5]:
start = time.time()

# 1. Column READY
for i in range(len(df.make_model)):
    selected = df.make_model[i].split()[1:]
    df['make_model'][i] = ' '.join(selected)

# 2. Column UNREADY
# df.short_description.sample(10)

# 3. Column READY
for i in range(len(df.make)):
    df['make'][i] = df.make[i].strip()

# 4. Column READY
def_clean_column(df.model)  

# 5. Column READY
for i in range(len(df.location)):
    if df.location[i] != 'Unknown':
        df.location[i] = df.location[i].split()[-1]
        
# 6. Column READY
for i in range(len(df.price)):
    df.price[i] = df.price[i].replace('€','').replace(',','').replace('.-','').strip()
df.price.replace('Unknown',np.nan, inplace=True)
df.price = df.price.astype('float')

# 7. Column READY
def_clean_column(df.body_type)

# 8. Column READY

def_clean_column(df.type)

# 9. Column READY
def_clean_column(df.doors)
df.doors.replace('Unknown',np.nan, inplace=True)
df.doors = df.doors.astype('float')

# 10. Column READY
def_clean_column(df.country_version)

# 11. Column READY
def_clean_column(df.offer_number)

# 12. Column READY
def_clean_column(df.warranty)

# 13. Column READY
for i in range(len(df.mileage)):
    df.mileage[i] = df.mileage[i].strip().replace(',','').replace('km','')
df.mileage.replace('Unknown',np.nan, inplace=True)
df.mileage = df.mileage.astype('float')

# 14. Column READY
df.first_registration.replace('Unknown', np.nan, inplace=True)
df.first_registration = pd.to_datetime(df.first_registration, format='%m/%Y')

# 15. Column READY
def_clean_column1(df.gearbox)

# 16. Column UNREADY
# df.fuel_type.value_counts()

# 17. Column READY
# df.colour.value_counts()

# 18. Column READY
# df.paint.value_counts()

# 19. Column UNREADY
for i in range(len(df.desc)):
    if type(df.desc[i]) == list:
        df.desc[i] = str(df.desc[i])
        
# 20. Column READY
df.seller.value_counts()

# 21. Column READY
def_clean_column(df.seats)
df.seats.replace('Unknown',np.nan, inplace=True)
df.seats = df.seats.astype('float')

# 22. Column READY
df['power_kw'] = pd.Series([None]*len(df))
df['power_hp'] = pd.Series([None]*len(df))
for i in range(len(df.power)):
    if df.power[i] != 'Unknown':
        df['power_kw'][i] = re.findall(r'\d+', df.power[i][0].strip())[0]
        df['power_hp'][i] = re.findall(r'\d+', df.power[i][0].strip())[1]
df.drop('power', axis=1, inplace=True)

# 23. Column READY
for i in range(len(df.engine_size)):
    if df.engine_size[i] != 'Unknown':
        df.engine_size[i] = df.engine_size[i][0].strip().replace(',','').rstrip('cc').strip()
df.engine_size.replace('Unknown',np.nan, inplace=True)
df.engine_size = df.engine_size.astype('float')

# 24. Column READY
def_clean_column1(df.gears)
df.gears.replace('Unknown',np.nan, inplace=True)
df.gears = df.gears.astype('float')

# 25. Column READY
for i in range(len(df.co2_emissions)):
    if df.co2_emissions[i] != 'Unknown':
        df.co2_emissions[i] = df.co2_emissions[i].replace('g/km (comb.)','').replace(',','').strip()
df.co2_emissions.replace('Unknown',np.nan, inplace=True)
df.co2_emissions = df.co2_emissions.astype('float')

# 26. Column UNREADY
# df.manufacturer_colour.value_counts()

# 27. Column READY
def_clean_column(df.drivetrain)

# 28. Column READY
def_clean_column1(df.cylinders)
df.cylinders.replace('Unknown',np.nan, inplace=True)
df.cylinders = df.cylinders.astype('float')

# 29. Column READY
df['consumption_comb'] = pd.Series([None]*len(df))
df['consumption_city'] = pd.Series([None]*len(df))
df['consumption_country'] = pd.Series([None]*len(df))

for i in range(len(df.fuel_consumption)):
    if df.fuel_consumption[i] != 'Unknown':
        if len(df.fuel_consumption[i]) == 1:
            assign_consumption_value(df, i, 0, df.fuel_consumption[i][0][0].split()[3])
        elif len(df.fuel_consumption[i]) == 2:
            assign_consumption_value(df, i, 0, df.fuel_consumption[i][0][0].split()[3])
            assign_consumption_value(df, i, 1, df.fuel_consumption[i][1][0].split()[3])
        elif len(df.fuel_consumption[i]) == 3:
            assign_consumption_value(df, i, 0, df.fuel_consumption[i][0][0].split()[3])
            assign_consumption_value(df, i, 1, df.fuel_consumption[i][1][0].split()[3])
            assign_consumption_value(df, i, 2, df.fuel_consumption[i][2][0].split()[3])
df.consumption_city = df.consumption_city.astype('float')
df.consumption_comb = df.consumption_comb.astype('float')
df.consumption_country = df.consumption_country.astype('float')
df.drop('fuel_consumption', axis=1, inplace=True)

# 30. Column READY
for i in range(len(df.comfort_convenience)):
    if df.comfort_convenience[i] != 'Unknown':
        df.comfort_convenience[i] = df.comfort_convenience[i][0]
        
# 31. Column READY
for i in range(len(df.entertainment_media)):
    if df.entertainment_media[i] != 'Unknown':
        df.entertainment_media[i] = df.entertainment_media[i][0]

# 32. Column READY
for i in range(len(df.safety_security)):
    if df.safety_security[i] != 'Unknown':
        df.safety_security[i] = df.safety_security[i][0]
        
# 33. Column READY
for i in range(len(df.extras)):
    if df.extras[i] != 'Unknown':
        df.extras[i] = df.extras[i][0]
        
# 34. Column READY
for i in range(len(df.empty_weight)):
    if df.empty_weight[i] != 'Unknown':
        df.empty_weight[i] = df.empty_weight[i][0].replace(',','').replace('kg','').strip()
df.empty_weight.replace('Unknown',np.nan, inplace=True)
df.empty_weight = df.empty_weight.astype('float')

# 35. Column READY
def_clean_column(df.model_code)

# 36. Column READY
# print(df.general_inspection.sample(20))

# 37. Column READY
# print(df.last_service.sample(20))

# 38. Column READY
# print(df.full_service_history.sample(20))

# 39. Column READY
# df.non_smoker_vehicle.value_counts()

# 40. Column READY
# df.emission_class.value_counts()

# 41. Column READY
# df.emissions_sticker.value_counts()

# 42. Column READY
# df.upholstery_colour.value_counts()

# 43. Column READY
# df.upholstery.value_counts()

# 44. Column READY
# df.production_date.value_counts()

# 45. Column READY
df['preown_km'] = pd.Series([None]*len(df))
df['preown_date'] = pd.Series([None]*len(df))
df['preown_num'] = pd.Series([None]*len(df))

for i in range(len(df.previous_owner)):
    if df.previous_owner[i] != 'Unknown':
        df.preown_km[i] = df.previous_owner[i][0][0].replace(',','').replace('km','').strip()
        df.preown_date[i] = df.previous_owner[i][0][1]
        df.preown_num[i] = df.previous_owner[i][1]
df.drop('previous_owner', axis=1, inplace=True)

# 46. Column READY
# df.other_fuel_types.value_counts()

# 47. Column READY
# df.power_consumption.value_counts()

# 48. Column UNREADY
# df.energy_efficiency_class.value_counts()

# 49. Column READY
# df.co2_efficiency.value_counts()

# 50. Column READY
# df.fuel_consumption_wltp.value_counts()

# 51. Column READY
# df.co2_emissions_wltp.value_counts()

# 52. Column READY

for i in range(len(df['available_from'])):
    if df['available_from'][i] != 'Unknown':
        df['available_from'][i] = df['available_from'][i][0].replace(',','').strip()
df.available_from.replace('Unknown', np.nan, inplace=True)
df.available_from = pd.to_datetime(df.available_from, format='%d/%m/%Y')

# 53. Column READY
# df.taxi_or_rental_car.value_counts()

# 54. Column READY
for i in range(len(df.availability)):
    if df.availability[i] != 'Unknown':
        df.availability[i] = df.availability[i][0].split()[2]
df.availability.replace('Unknown',np.nan, inplace=True)
df.availability = df.availability.astype('float')

# 55. Column READY
# df.last_timing_belt_change.value_counts()

# 56. Column READY
# df.electric_range_wltp.value_counts()

# 57. Column READY
# df.power_consumption_wltp.value_counts()

# 58. Column READY
# df.battery_ownership.value_counts()

# ADDITIONAL
df['fuel_type'] = df['fuel_type'].transform(lambda x: 'Benzine' if re.search('Gasoline', x) else 'Benzine' if re.search('Super', x) else 'Benzine' if re.search('Benzine', x) else 'Benzine' if re.search('Others', x) else 'LPG' if re.search('CNG', x) else 'LPG' if re.search('LPG', x) else 'LPG' if re.search('gas', x) else 'Electric' if re.search('Electric', x) else 'LPG' if re.search('Ethanol', x) else 'LPG' if re.search('Biogas', x) else 'Diesel' if re.search('Diesel', x) else 'Diesel')
for i in range(len(df)):
    if df.iloc[i]['other_fuel_types'] == 'Electricity':
        df['fuel_type'].iloc[i] = 'Electric'

df.power_hp = df.power_hp.astype('float')
df.power_kw = df.power_kw.astype('float')
        

df.replace('Unknown', np.nan, inplace=True)
df.drop_duplicates(keep='first', inplace=True)
df.reset_index(drop=True, inplace=True)
df.drop(1775, axis=0, inplace=True)

for i in df[(df.model == '') & (df.make=='Opel')].index:
    a = df.short_description[i].split()[0]
    if len(re.findall('\d', a)) > 0:
        pass
    else:
        df.model[i] = a
for i in df[df.model == ''].index:
    df.model[i] = np.nan
for i in df[(df.consumption_city.isnull() == False) & (df.consumption_country.isnull() == False) & (df.consumption_comb.isnull() == True)].index:
    df.consumption_comb[i] = (df.consumption_city[i] + df.consumption_country[i])/2
    

end = time.time()

print("Kodun çalışma süresi: {:.2f} saniye".format(end - start))
# Kodun çalışma süresi: 157.99 saniye
# Kodun çalışma süresi: 204.40 saniye

Kodun çalışma süresi: 172.18 saniye


In [6]:
df_clean = df.copy()

In [7]:
df_clean.to_csv('df_clean.csv', index=False)

# PART- 2 `( Handling With Missing Vales )`

In [8]:
start = time.time()

def_fill_miss(df, 1, 'mode', 'doors', 'body_type')

def_fill_miss(df, 1, 'mean', 'first_registration', 'model')
df.first_registration = df.first_registration.transform(lambda x: x.year)

def_fill_miss(df, 3, 'mode', 'model', 'make', 'first_registration', 'body_type')

def_fill_miss(df, 2, 'mean', 'mileage', 'model', 'first_registration')

def_fill_miss(df, 2, 'mode', 'gearbox', 'model', 'first_registration')

def_fill_miss(df, 2, 'mode', 'seats', 'model', 'body_type')

def_fill_miss(df, 2, 'mode', 'cylinders', 'make', 'model', 'first_registration')

def_fill_miss(df, 3, 'mode', 'engine_size', 'cylinders', 'first_registration', 'fuel_type')

def_fill_miss(df, 2, 'mode', 'gears', 'model', 'first_registration')

def_fill_miss(df, 2, 'mode', 'drivetrain', 'model', 'first_registration')

def_fill_miss(df, 2, 'mode', 'comfort_convenience', 'first_registration', 'model')

def_fill_miss(df, 2, 'mode', 'entertainment_media', 'first_registration', 'model')

def_fill_miss(df, 2, 'mode', 'safety_security', 'first_registration', 'model')

def_fill_miss(df, 2, 'mode', 'extras', 'model', 'first_registration')

def_fill_miss(df, 3, 'mode', 'empty_weight', 'model', 'first_registration', 'body_type')

def_fill_miss(df, 2, 'mode', 'colour', 'model', 'body_type')

def_fill_miss(df, 3, 'mode', 'emission_class', 'model', 'engine_size', 'body_type')

def_fill_miss(df, 3, 'mean', 'consumption_comb', 'make', 'first_registration', 'cylinders')

def_fill_miss(df, 3, 'mode', 'upholstery','make', 'first_registration', 'body_type')

def_fill_miss(df, 3, 'mean', 'power_hp', 'model', 'engine_size', 'first_registration')

def_fill_miss(df, 3, 'mean', 'power_kw', 'model', 'engine_size', 'first_registration')

end = time.time()

print("\nKodun çalışma süresi: {:.2f} saniye".format(end - start))

# Kodun çalışma süresi: 603.73 saniye
# Kodun çalışma süresi: 469.59 saniye
# Kodun çalışma süresi: 426.79 saniye


Kodun çalışma süresi: 430.93 saniye


In [9]:
df_miss = df.copy()

In [10]:
df_miss.to_csv('df_miss.csv', index=False)

# PART- 3 `( Handling With Outliers )`

In [11]:
df = df[df.seats <= 5]

In [12]:
df = df[df.first_registration <= 2022]

In [13]:
df = df[(df.gears >= 4) & (df.gears <=10)]

In [14]:
df = df[(df.cylinders >= 2) & (df.cylinders <= 8)]

In [15]:
df = df[(df.engine_size >= 800) & (df.cylinders <= 6000)]

In [16]:
df = df[(df.empty_weight >= 700) & (df.empty_weight <= 25000)]

# Final Step (Checking final situation of data via graphs)

In [17]:
df.dropna(axis=1, inplace=True)
df.drop_duplicates(keep='first', inplace=True)
df.reset_index(drop=True, inplace=True)

In [18]:
df_final = df.copy()

In [19]:
df_final.to_csv('df_final.csv', index=False)

## Export dataframe to csv file (without dummy)

In [20]:
for i in range(len(df)):
    if df.first_registration[i] > 2017:
        df.model[i] = 'New'
    else:
        df.model[i] = 'Old'
        
for i in range(len(df)):
    if df.colour[i] in ['Black', 'Grey', 'Silver', 'Bronze', 'Brown']:
        df.colour[i] = 'Dark'
    elif df.colour[i] in ['Yellow', 'White', 'Beige','Gold']:
        df.colour[i] = 'Light'
    else:
        df.colour[i] = 'Others' 
        
for i in range(len(df)):
    if df.seller[i] != 'Dealer':
        df.seller[i] = 'Private_Seller'        
        
for i in range(len(df)):
    df.comfort_convenience[i] = len(df.comfort_convenience[i].split(','))        
        
for i in range(len(df)):
    df.entertainment_media[i] = len(df.entertainment_media[i].split(','))        
        
for i in range(len(df)):
    df.safety_security[i] = len(df.safety_security[i].split(','))        
        
for i in range(len(df)):
    df.extras[i] = len(df.extras[i].split(','))        

In [21]:
df.first_registration = df.first_registration.transform(lambda x: 2023-x)

In [22]:
df.drop_duplicates(keep='first', inplace=True)
df.reset_index(drop=True, inplace=True)

In [23]:
df.drop('short_description', axis=1, inplace=True)
df.drop('make_model', axis=1, inplace=True)

In [24]:
df.comfort_convenience = df.comfort_convenience.astype('int')
df.entertainment_media = df.entertainment_media.astype('int')
df.safety_security = df.safety_security.astype('int')
df.extras = df.extras.astype('int')

In [25]:
df.drop_duplicates(keep='first', inplace=True)
df.reset_index(drop=True, inplace=True)

In [26]:
df.to_csv("final_scout_not_dummy.csv", index=False)

# Dummy Operation

## Export dataframe to csv file (dummy)

In [27]:
df_dummied = pd.get_dummies(df)

In [28]:
df_dummied.to_csv("final_scout_dummy.csv", index=False)